## Steps:
### 1. Find the MAX Fiscal Period (Fiscal Month) coming from FINBI data file
### 2. Find the month that is eleven months back from that max month - plus a 30-day window, so effectively 12 months back from the current month
### 3. Find the MIN (earlest) date for that Step2 month back - this becomes the earlest readout date to query from the Compass tracker data

In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import os
from compass_smartsheet import CompassSmartsheet
import cisco_fiscal_calendar as fc

In [3]:
finbi_path = r'C:\Users\phsheari\Documents\FIN BI Data\FINBI_Bookings_Accrued_Results.pkl'
fins = pd.read_pickle(finbi_path)

In [4]:
fins['Fiscal Month'] = fins['Fiscal Week ID'].apply(lambda x: int(str(x)[:6]))

In [5]:
month_list = list(fins['Fiscal Month'].unique())

In [6]:
month_list.sort()

In [7]:
earliest_finbi_month = month_list[11] # we need 11 months plus a 30-day minimum booking window, so 12 months back
last_finbi_month = month_list[-1]

In [8]:
earliest_finbi_month, last_finbi_month

(202012, 202201)

In [9]:
A1 = month_list.index(earliest_finbi_month)
A2 = month_list.index(last_finbi_month)

In [10]:
use_list = month_list[A1:]

In [11]:
use_list

[202012,
 202101,
 202102,
 202103,
 202104,
 202105,
 202106,
 202107,
 202108,
 202109,
 202110,
 202111,
 202112,
 202201]

In [4]:
compass_tracker_pickle_version = r'C:\Users\phsheari\Documents\SFDC Data\PIPELINE UPDATE\SFDC_Trackerfile.pkl'

In [5]:
tracker = pd.read_pickle(compass_tracker_pickle_version)

In [6]:
current_run_date = dt.datetime.today() #.isoformat()[:10]

In [7]:
file_run_date = current_run_date.isoformat()[:10]

In [8]:
thirty_days_ago = current_run_date + dt.timedelta(weeks = -4, days = -2)

In [9]:
thirty_days_ago = thirty_days_ago.isoformat()[:10]

In [10]:
thirty_days_ago

'2021-07-06'

In [11]:
eleven_months_ago = current_run_date + dt.timedelta(weeks = -48, days = -3)

In [12]:
eleven_months_ago = eleven_months_ago.isoformat()[:10]

In [13]:
eleven_months_ago

'2020-08-31'

In [14]:
# eleven_months_ago = today + dt.timedelta(weeks = -48, days = -3)

In [15]:
# eleven_months_ago = eleven_months_ago.isoformat()[:10]

In [16]:
caldf = fc.calendar_df

In [17]:
oldest_fiscal_week_id = caldf.loc[caldf['date'] == eleven_months_ago]['finbi_fiscal_week_id'].values[0]

In [18]:
oldest_fiscal_week_id

2021022

In [19]:
current_run_date_str = current_run_date.isoformat()[:10]

In [20]:
current_fiscal_week_id = caldf.loc[caldf['date'] == current_run_date_str]['finbi_fiscal_week_id'].values[0]

In [21]:
current_fiscal_week_id

2022011

In [22]:
thirty_days_ago_week_id = caldf.loc[caldf['date'] == thirty_days_ago]['finbi_fiscal_week_id'].values[0]

In [23]:
thirty_days_ago_week_id

2021122

In [24]:
fins['FINBI Name'] = fins.loc[fins['CAV Name'].notnull()].apply(lambda x: x['GUID Name'] + '_' + x['CAV Name'], axis=1)

In [25]:
fins.loc[fins['Sales Level 2']=='GLOBAL ENTERPRISE SEGMENT','Sales Level 2']='GES'

In [26]:
str(current_fiscal_week_id)[:6]

'202201'

In [27]:
fiscal_month_id = str(current_fiscal_week_id)[:6]

In [28]:
fins['Report Month'] = fiscal_month_id

In [29]:
#((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id))

In [30]:
fin_sl1           = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) & ~(fins['Sales Level 2']=='LATIN AMERICA')].groupby(['Sales Level 1'])[['FINBI Name']].nunique()
fin_ct            = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) & ~(fins['Sales Level 2']=='LATIN AMERICA')].groupby(['Campaign Type'])[['FINBI Name']].nunique()
fin_sl1_ct        = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) & ~(fins['Sales Level 2']=='LATIN AMERICA')].groupby(['Sales Level 1','Campaign Type'])[['FINBI Name']].nunique()
fin_sl1_sl2_ct    = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) & ~(fins['Sales Level 2']=='LATIN AMERICA')].groupby(['Sales Level 1','Sales Level 2','Campaign Type'])[['FINBI Name']].nunique()
fin_sl1_sl2       = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) & ~(fins['Sales Level 2']=='LATIN AMERICA')].groupby(['Sales Level 1','Sales Level 2'])[['FINBI Name']].nunique()
overall_numerator = fins.loc[(fins['CAV Name'].notnull()) & ((fins['Fiscal Week ID'] >= oldest_fiscal_week_id) & (fins['Fiscal Week ID'] <= thirty_days_ago_week_id)) & ~(fins['Sales Level 2']=='LATIN AMERICA')][['FINBI Name']].nunique()[0]

In [31]:
fin_sl1_ct.reset_index(inplace=True)
fin_ct.reset_index(inplace=True)
fin_sl1.reset_index(inplace=True)
fin_sl1_sl2_ct.reset_index(inplace=True)
fin_sl1_sl2.reset_index(inplace=True)

In [32]:
overall_numerator

573

### Set the SL and Campaign Dimension Level on the various dataframes

In [33]:
fin_sl1_ct['Report Month'] = fiscal_month_id
fin_sl1_ct['Dimension Grouping'] = 'SL1_CT'

In [34]:
fin_ct['Report Month'] = fiscal_month_id
fin_ct['Dimension Grouping'] = 'CT ONLY'

In [35]:
fin_sl1['Report Month'] = fiscal_month_id
fin_sl1['Dimension Grouping'] = 'SL1 ONLY'

In [36]:
fin_sl1_sl2_ct['Report Month'] = fiscal_month_id
fin_sl1_sl2_ct['Dimension Grouping'] = 'SL1_SL2_CT'

In [37]:
fin_sl1_sl2['Report Month'] = fiscal_month_id
fin_sl1_sl2['Dimension Grouping'] = 'SL1_SL2'

In [38]:
fin_sl1_sl2_ct

,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Report Month,Dimension Grouping
0,APJC,ANZ AREA,PROACTIVE,7,202201,SL1_SL2_CT
1,APJC,ANZ AREA,REACTIVE,6,202201,SL1_SL2_CT
2,APJC,APJ_SP,PROACTIVE,10,202201,SL1_SL2_CT
3,APJC,APJ_SP,REACTIVE,3,202201,SL1_SL2_CT
4,APJC,ASEAN_AREA,PROACTIVE,8,202201,SL1_SL2_CT
5,APJC,ASEAN_AREA,REACTIVE,4,202201,SL1_SL2_CT
6,APJC,GREATER_CHINA,PROACTIVE,2,202201,SL1_SL2_CT
7,APJC,GREATER_CHINA,REACTIVE,3,202201,SL1_SL2_CT
8,APJC,INDIA_AREA,PROACTIVE,6,202201,SL1_SL2_CT
9,APJC,INDIA_AREA,REACTIVE,4,202201,SL1_SL2_CT


In [39]:
fin_sl1['Sales Level 1']        = fin_sl1['Sales Level 1'].str.upper()
fin_ct['Campaign Type']         = fin_ct['Campaign Type'].str.upper()
fin_sl1_ct['Sales Level 1']     = fin_sl1_ct['Sales Level 1'].str.upper()
fin_sl1_sl2_ct['Sales Level 1'] = fin_sl1_sl2_ct['Sales Level 1'].str.upper()
fin_sl1_sl2_ct['Sales Level 2'] = fin_sl1_sl2_ct['Sales Level 2'].str.upper()
fin_sl1_sl2['Sales Level 1']    = fin_sl1_sl2['Sales Level 1'].str.upper()
fin_sl1_sl2['Sales Level 2']    = fin_sl1_sl2['Sales Level 2'].str.upper()

In [40]:
removed_campaigns = ['REACTIVE - ACC DESIGNATION','REACTIVE - PARTNER','REACTIVE - GROWTH INITIATIVE',
                     'PROACTIVE - TAC LEAD','PROACTIVE - CS RESEARCH','PROACTIVE - PRESALES ALIGN PITSTOP EFFORT']

In [41]:
tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('UKI', case=False)), 'Lvl2 (Region)'] = 'EMEAR-UKI'

In [42]:
tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('central', case=False)), 'Lvl2 (Region)'] = 'EMEAR-CENTRAL'

In [43]:
tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('north', case=False)), 'Lvl2 (Region)'] = 'EMEAR-NORTH'

In [44]:
tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('south', case=False)), 'Lvl2 (Region)'] = 'EMEAR-SOUTH'

In [45]:
tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('ANZ', case=False)), 'Lvl2 (Region)'] = 'ANZ AREA'

In [46]:
tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('japan', case=False)), 'Lvl2 (Region)'] = 'JAPAN__'

In [47]:
trkr_condition1A = (tracker['Date Readout Done']>= eleven_months_ago)
trkr_condition1B = (tracker['Date Readout Done']<= thirty_days_ago)
trkr_condition2 = ~(tracker['Compass Campaign Name'].isin(removed_campaigns))
trkr_condition3 = ~(tracker['Readout Delivery Status'].isin(['Declined','Postponed']))
trkr_condition4 = ~(tracker['Lvl2 (Region)']=='LATAM')

In [48]:
compass_sl1         = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4].groupby(['Lvl1'])[['Customer Name']].nunique()
compass_ct          = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4].groupby(['Campaign Type'])[['Customer Name']].nunique()
compass_sl1_ct      = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4].groupby(['Lvl1','Campaign Type'])[['Customer Name']].nunique()
compass_sl1_sl2_ct  = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4].groupby(['Lvl1','Lvl2 (Region)','Campaign Type'])[['Customer Name']].nunique()
compass_sl1_sl2     = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4].groupby(['Lvl1','Lvl2 (Region)'])[['Customer Name']].nunique()
overall_denominator = tracker.loc[trkr_condition1A & trkr_condition1B & trkr_condition2 & trkr_condition3 & trkr_condition4][['Customer Name']].nunique()[0]

In [49]:
# ### Sanity Check
# tracker.loc[(tracker['Lvl2 (Region)'].notnull()) & (tracker['Lvl2 (Region)'].str.contains('UKI', case=False)), 'Lvl2 (Region)'] = 'EMEAR-UKI'

In [50]:
compass_sl1.reset_index(inplace=True)
compass_ct.reset_index(inplace=True)
compass_sl1_ct.reset_index(inplace=True)
compass_sl1_sl2_ct.reset_index(inplace=True)
compass_sl1_sl2.reset_index(inplace=True)

In [51]:
compass_sl1.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
compass_ct.rename(columns = {'Customer Name':'Compass Name'}, inplace=True)
compass_sl1_ct.rename(columns = {'Lvl1':'Sales Level 1','Customer Name':'Compass Name'}, inplace=True)
compass_sl1_sl2_ct.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Customer Name':'Compass Name'}, inplace=True)
compass_sl1_sl2.rename(columns = {'Lvl1':'Sales Level 1','Lvl2 (Region)':'Sales Level 2','Customer Name':'Compass Name'}, inplace=True)

In [52]:
fin_sl1_sl2_ct

,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Report Month,Dimension Grouping
0,APJC,ANZ AREA,PROACTIVE,7,202201,SL1_SL2_CT
1,APJC,ANZ AREA,REACTIVE,6,202201,SL1_SL2_CT
2,APJC,APJ_SP,PROACTIVE,10,202201,SL1_SL2_CT
3,APJC,APJ_SP,REACTIVE,3,202201,SL1_SL2_CT
4,APJC,ASEAN_AREA,PROACTIVE,8,202201,SL1_SL2_CT
5,APJC,ASEAN_AREA,REACTIVE,4,202201,SL1_SL2_CT
6,APJC,GREATER_CHINA,PROACTIVE,2,202201,SL1_SL2_CT
7,APJC,GREATER_CHINA,REACTIVE,3,202201,SL1_SL2_CT
8,APJC,INDIA_AREA,PROACTIVE,6,202201,SL1_SL2_CT
9,APJC,INDIA_AREA,REACTIVE,4,202201,SL1_SL2_CT


In [53]:
combined_sl1 = pd.merge(fin_sl1, compass_sl1, on = ['Sales Level 1'])

In [54]:
combined_ct = pd.merge(fin_ct, compass_ct, on=['Campaign Type'])

In [55]:
combined_sl1_ct = pd.merge(fin_sl1_ct, compass_sl1_ct, on = ['Sales Level 1','Campaign Type'])

In [56]:
combined_sl1_sl2_ct = pd.merge(fin_sl1_sl2_ct,compass_sl1_sl2_ct, on = ['Sales Level 1', 'Sales Level 2','Campaign Type'])

In [57]:
combined_sl1_sl2 = pd.merge(fin_sl1_sl2,compass_sl1_sl2, on = ['Sales Level 1', 'Sales Level 2'])

In [58]:
combined_sl1_sl2_ct

,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Report Month,Dimension Grouping,Compass Name
0,APJC,ANZ AREA,PROACTIVE,7,202201,SL1_SL2_CT,66
1,APJC,ANZ AREA,REACTIVE,6,202201,SL1_SL2_CT,17
2,APJC,APJ_SP,PROACTIVE,10,202201,SL1_SL2_CT,27
3,APJC,APJ_SP,REACTIVE,3,202201,SL1_SL2_CT,4
4,APJC,ASEAN_AREA,PROACTIVE,8,202201,SL1_SL2_CT,34
5,APJC,ASEAN_AREA,REACTIVE,4,202201,SL1_SL2_CT,5
6,APJC,GREATER_CHINA,PROACTIVE,2,202201,SL1_SL2_CT,83
7,APJC,GREATER_CHINA,REACTIVE,3,202201,SL1_SL2_CT,7
8,APJC,INDIA_AREA,PROACTIVE,6,202201,SL1_SL2_CT,39
9,APJC,INDIA_AREA,REACTIVE,4,202201,SL1_SL2_CT,7


In [59]:
combined_sl1_sl2_ct['Conversion Ratio %'] = (combined_sl1_sl2_ct['FINBI Name'] / combined_sl1_sl2_ct['Compass Name']) * 100
combined_sl1_ct['Conversion Ratio %'] = (combined_sl1_ct['FINBI Name'] / combined_sl1_ct['Compass Name']) * 100
combined_sl1['Conversion Ratio %'] = (combined_sl1['FINBI Name'] / combined_sl1['Compass Name']) * 100
combined_ct['Conversion Ratio %'] = (combined_ct['FINBI Name'] / combined_ct['Compass Name']) * 100
combined_sl1_sl2['Conversion Ratio %'] = (combined_sl1_sl2['FINBI Name'] / combined_sl1_sl2['Compass Name']) * 100

In [60]:
combined_ct['Sales Level 1'] = pd.NA
combined_ct['Sales Level 2'] = pd.NA

In [61]:
combined_ct = combined_ct[['Dimension Grouping', 'Report Month','Sales Level 1','Sales Level 2','Campaign Type', 'FINBI Name',  'Compass Name','Conversion Ratio %']]

In [62]:
combined_sl1_ct['Sales Level 2'] = pd.NA

In [63]:
combined_sl1_ct = combined_sl1_ct[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [64]:
combined_sl1['Sales Level 2'] = pd.NA
combined_sl1['Campaign Type'] = pd.NA

In [65]:
combined_sl1 = combined_sl1[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2','Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [66]:
combined_sl1_sl2_ct = combined_sl1_sl2_ct[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [67]:
combined_sl1_sl2['Campaign Type'] = pd.NA

In [68]:
combined_sl1_sl2 = combined_sl1_sl2[['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2', 'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %']]

In [69]:
combined_ct

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,CT ONLY,202201,<NA>,<NA>,PROACTIVE,300,1553,19.317450
1,CT ONLY,202201,<NA>,<NA>,REACTIVE,311,1111,27.992799


In [70]:
combined_sl1

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,SL1 ONLY,202201,APJC,<NA>,<NA>,64,394,16.243655
1,SL1 ONLY,202201,AMERICAS,<NA>,<NA>,421,1635,25.749235
2,SL1 ONLY,202201,EMEAR-REGION,<NA>,<NA>,89,477,18.658281


In [71]:
combined_sl1_ct

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,SL1_CT,202201,APJC,<NA>,PROACTIVE,48,351,13.675214
1,SL1_CT,202201,APJC,<NA>,REACTIVE,20,50,40.000000
2,SL1_CT,202201,AMERICAS,<NA>,PROACTIVE,195,875,22.285714
3,SL1_CT,202201,AMERICAS,<NA>,REACTIVE,258,895,28.826816
4,SL1_CT,202201,EMEAR-REGION,<NA>,PROACTIVE,57,330,17.272727
5,SL1_CT,202201,EMEAR-REGION,<NA>,REACTIVE,33,169,19.526627


In [72]:
combined_sl1_sl2

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,SL1_SL2,202201,APJC,ANZ AREA,<NA>,13,79,16.455696
1,SL1_SL2,202201,APJC,APJ_SP,<NA>,11,31,35.483871
2,SL1_SL2,202201,APJC,ASEAN_AREA,<NA>,11,39,28.205128
3,SL1_SL2,202201,APJC,GREATER_CHINA,<NA>,5,89,5.617978
4,SL1_SL2,202201,APJC,INDIA_AREA,<NA>,9,46,19.565217
5,SL1_SL2,202201,APJC,JAPAN__,<NA>,14,96,14.583333
6,SL1_SL2,202201,APJC,ROK_AREA,<NA>,3,16,18.750000
7,SL1_SL2,202201,AMERICAS,AMERICAS_SP,<NA>,10,88,11.363636
8,SL1_SL2,202201,AMERICAS,CANADA,<NA>,28,119,23.529412
9,SL1_SL2,202201,AMERICAS,GES,<NA>,160,376,42.553191


In [73]:
combined_sl1_sl2_ct

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,SL1_SL2_CT,202201,APJC,ANZ AREA,PROACTIVE,7,66,10.606061
1,SL1_SL2_CT,202201,APJC,ANZ AREA,REACTIVE,6,17,35.294118
2,SL1_SL2_CT,202201,APJC,APJ_SP,PROACTIVE,10,27,37.037037
3,SL1_SL2_CT,202201,APJC,APJ_SP,REACTIVE,3,4,75.000000
4,SL1_SL2_CT,202201,APJC,ASEAN_AREA,PROACTIVE,8,34,23.529412
5,SL1_SL2_CT,202201,APJC,ASEAN_AREA,REACTIVE,4,5,80.000000
6,SL1_SL2_CT,202201,APJC,GREATER_CHINA,PROACTIVE,2,83,2.409639
7,SL1_SL2_CT,202201,APJC,GREATER_CHINA,REACTIVE,3,7,42.857143
8,SL1_SL2_CT,202201,APJC,INDIA_AREA,PROACTIVE,6,39,15.384615
9,SL1_SL2_CT,202201,APJC,INDIA_AREA,REACTIVE,4,7,57.142857


### Campaign Type Only

In [74]:
combined_ct.columns

Index(['Dimension Grouping', 'Report Month', 'Sales Level 1', 'Sales Level 2',
       'Campaign Type', 'FINBI Name', 'Compass Name', 'Conversion Ratio %'],
      dtype='object')

### Sales Level 1 Only

In [75]:
combined_sl1

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,SL1 ONLY,202201,APJC,<NA>,<NA>,64,394,16.243655
1,SL1 ONLY,202201,AMERICAS,<NA>,<NA>,421,1635,25.749235
2,SL1 ONLY,202201,EMEAR-REGION,<NA>,<NA>,89,477,18.658281


### SL1 = Sales Level 1 and CT = Campaign Type

In [76]:
combined_sl1_ct

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,SL1_CT,202201,APJC,<NA>,PROACTIVE,48,351,13.675214
1,SL1_CT,202201,APJC,<NA>,REACTIVE,20,50,40.000000
2,SL1_CT,202201,AMERICAS,<NA>,PROACTIVE,195,875,22.285714
3,SL1_CT,202201,AMERICAS,<NA>,REACTIVE,258,895,28.826816
4,SL1_CT,202201,EMEAR-REGION,<NA>,PROACTIVE,57,330,17.272727
5,SL1_CT,202201,EMEAR-REGION,<NA>,REACTIVE,33,169,19.526627


In [77]:
# with pd.ExcelWriter(r'C:\Users\phsheari\Documents\FIN BI Data\Conversion Rate\ConversionRates_' + file_run_date + '_.xlsx') as writer:
#     combined_sl1_ct.to_excel(writer, sheet_name = 'Theater_CampaignType', index=False)
#     combined_sl1_sl2_ct.to_excel(writer, sheet_name = 'Theater_SL2_CampaignType', index=False)
#     combined_sl1.to_excel(writer, sheet_name = 'Theater_Only', index=False)
#     combined_ct.to_excel(writer, sheet_name = 'CampaignType_Only', index=False)      

### SL1 = Sales Level 1 and SL2 = Sales Level 2 and CT = Campaign Type

In [78]:
# combined_sl1_sl2_ct

In [79]:
# combined_sl1_sl2_ct.to_excel(r'C:\Users\phsheari\Documents\Francisco\USCOMMERCIAL_ConversionRates.xlsx', index=False)

In [80]:
overall_conversion_ratio = (overall_numerator / overall_denominator) * 100
print(overall_numerator, overall_denominator)

573 2498


In [81]:
overall_list = [['OVERALL',fiscal_month_id, pd.NA, pd.NA, pd.NA, overall_numerator, overall_denominator, overall_conversion_ratio]]
overall_columns = ['Dimension Grouping', 'Report Month','Sales Level 1','Sales Level 2','Campaign Type','FINBI Name','Compass Name','Conversion Ratio %']
overall = pd.DataFrame(data = overall_list, columns = overall_columns)

In [82]:
overall

,Dimension Grouping,Report Month,Sales Level 1,Sales Level 2,Campaign Type,FINBI Name,Compass Name,Conversion Ratio %
0,OVERALL,202201,<NA>,<NA>,<NA>,573,2498,22.938351


In [83]:
overall_conversion_ratio

22.938350680544435

In [84]:
load_table = pd.concat([overall,combined_ct,combined_sl1, combined_sl1_ct, combined_sl1_sl2, combined_sl1_sl2_ct])

In [87]:
load_table['Report Month'] = '202112'

In [88]:
load_table.to_excel(r'C:\Users\phsheari\Documents\FIN BI Data\Conversion Rate\ConversionRates_' + fiscal_month_id + '_.xlsx', sheet_name=fiscal_month_id, index=False)

In [89]:
load_table['Sales Level 2'].unique()

array([<NA>, 'ANZ AREA', 'APJ_SP', 'ASEAN_AREA', 'GREATER_CHINA',
       'INDIA_AREA', 'JAPAN__', 'ROK_AREA', 'AMERICAS_SP', 'CANADA',
       'GES', 'US COMMERCIAL', 'US PS MARKET SEGMENT', 'EMEAR-CENTRAL',
       'EMEAR-NORTH', 'EMEAR-SOUTH', 'EMEAR-UKI', 'EMEAR_GERMANY',
       'EMEAR_MEA', 'EMEAR_SP'], dtype=object)

In [ ]:
load_table.loc[load_table['Sales Level 2'] == 'US PS MARKET SEGMENT']

In [ ]:
len(sorted(list(fins['Fiscal Week ID'].unique())))

In [ ]:
sorted(list(fins['Fiscal Week ID'].unique()))[0:37]

In [ ]:
2021012 - 2020013